<a href="https://colab.research.google.com/github/gzguevara/amex_kaggle/blob/master/Train_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Drive and XGB

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!pip uninstall xgboost
!pip install xgboost

#!pip install import_ipynb
#import import_ipynb

Mounted at /content/drive
Found existing installation: xgboost 0.90
Uninstalling xgboost-0.90:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/xgboost-0.90.dist-info/*
    /usr/local/lib/python3.7/dist-packages/xgboost/*
    /usr/local/xgboost/libxgboost.so
Proceed (y/n)? y
  Successfully uninstalled xgboost-0.90
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 192.9 MB 54 kB/s 


# Imports

In [ ]:
import pandas as pd
import numpy  as np  
import cupy
# for cupu memory managmet
mempool        = cupy.get_default_memory_pool()
pinned_mempool = cupy.get_default_pinned_memory_pool()

import xgboost as xgb
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split

import sys
from importlib import reload #test = reload(test)

import matplotlib.pyplot as plt, gc, os
import plotly.express as px
import seaborn as sns

import random
import itertools

# Paths
train_dir   = '/content/drive/MyDrive/KaggleAMEX/Data/train.parquet'
impor_dir   = '/content/drive/MyDrive/KaggleAMEX/Importance'
targets_dir = '/content/drive/MyDrive/KaggleAMEX/Data/train_labels.csv'

model_dir   = '/content/drive/MyDrive/KaggleAMEX/Models/freq_1'
#Parameters
SEED  = 42
FOLDS = 5

#Get numerical and catecorical columns 
cat_features = ['B_30', 'B_38', 'D_117', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68', 'R_2', 'S_6', 'R_4', 'R_15', 'S_18', 'D_86', 'D_87', 'B_31', 'R_19', 'B_32', 'S_20', 'R_21', 'R_22', 'R_23', 'D_93', 'D_94', 'R_24', 'R_25', 'D_96', 'D_127', 'R_28', 'B_33', 'D_92', 'D_103', 'R_27', 'D_109', 'D_114', 'D_116', 'D_120', 'D_129', 'D_135', 'D_137', 'D_139', 'D_140', 'D_143']
num_features = ['P_2', 'D_39', 'B_1', 'B_2', 'R_1', 'S_3', 'D_41', 'B_3', 'D_42', 'D_43', 'D_44', 'B_4', 'D_45', 'B_5', 'D_46', 'D_47', 'D_48', 'D_49', 'B_6', 'B_7', 'B_8', 'D_50', 'D_51', 'B_9', 'R_3', 'D_52', 'P_3', 'B_10', 'D_53', 'S_5', 'B_11', 'D_54', 'S_7', 'B_12', 'S_8', 'D_55', 'D_56', 'B_13', 'R_5', 'D_58', 'S_9', 'B_14', 'D_59', 'D_60', 'D_61', 'B_15', 'S_11', 'D_62', 'D_65', 'B_16', 'B_17', 'B_18', 'B_19', 'B_20', 'S_12', 'R_6', 'S_13', 'B_21', 'D_69', 'B_22', 'D_70', 'D_71', 'D_72', 'S_15', 'B_23', 'D_73', 'P_4', 'D_74', 'D_75', 'D_76', 'B_24', 'R_7', 'D_77', 'B_25', 'B_26', 'D_78', 'D_79', 'R_8', 'R_9', 'S_16', 'D_80', 'R_10', 'R_11', 'B_27', 'D_81', 'D_82', 'S_17', 'R_12', 'B_28', 'R_13', 'D_83', 'R_14', 'D_84', 'R_16', 'B_29', 'R_17', 'R_18', 'D_88', 'S_19', 'R_20', 'D_89', 'D_91', 'S_22', 'S_23', 'S_24', 'S_25', 'S_26', 'D_102', 'D_104', 'D_105', 'D_106', 'D_107', 'B_36', 'B_37', 'R_26', 'D_108', 'D_110', 'D_111', 'B_39', 'D_112', 'B_40', 'S_27', 'D_113', 'D_115', 'D_118', 'D_119', 'D_121', 'D_122', 'D_123', 'D_124', 'D_125', 'D_128', 'B_41', 'B_42', 'D_130', 'D_131', 'D_132', 'D_133', 'D_134', 'D_136', 'D_138', 'D_141', 'D_142', 'D_144', 'D_145']


# Data

In [ ]:
'''
if cus_range == [1]:

      features =  [feature for feature in df.columns if 'last' in feature]
      features += [feature for feature in df.columns if 'sum_na' in feature] + ['target']
      df       =  df.loc[customers, features]
    
    elif cus_range == [2]:    df = df.loc[customers]
    elif cus_range == [3]:    pass
    elif cus_range == [4,12]: pass
    else: pass
'''

"\nif cus_range == [1]:\n\n      features =  [feature for feature in df.columns if 'last' in feature]\n      features += [feature for feature in df.columns if 'sum_na' in feature] + ['target']\n      df       =  df.loc[customers, features]\n    \n    elif cus_range == [2]:    df = df.loc[customers]\n    elif cus_range == [3]:    pass\n    elif cus_range == [4,12]: pass\n    else: pass\n"

In [ ]:
def process_and_feature_engineer(cus_range):

    # 13
    frequency = pd.read_csv('/content/drive/MyDrive/KaggleAMEX/Data/frequency.csv', index_col=0)
    customers = frequency.loc[frequency.frequency.isin(cus_range)].index.values
    
    # num - mean, std, min, max, last, first  cat - last, unique
    dir = '/content/drive/MyDrive/KaggleAMEX/Data/base_variables.parquet'
    df  = pd.read_parquet(dir)
    df  = df.loc[customers]
    # replace categorial
    dir = '/content/drive/MyDrive/KaggleAMEX/Data/base_variables_cat_woe.parquet'
    new = pd.read_parquet(dir)
    new = new.loc[customers]
    df[new.columns] = new
    print('Done base', df.shape)
    
    # Payment variables
    dir = '/content/drive/MyDrive/KaggleAMEX/Data/payment_variables.parquet'
    new = pd.read_parquet(dir)
    new = new.loc[customers]
    df  = pd.concat([new, df], axis=1)
    print('Done payment variables', df.shape)
    
    # Risk variables 
    dir = '/content/drive/MyDrive/KaggleAMEX/Data/risk_variables.parquet'
    new = pd.read_parquet(dir)
    new = new.loc[customers]
    df  = pd.concat([new, df], axis=1)
    print('Done risk variables', df.shape)
    
    # Spend variables 
    dir = '/content/drive/MyDrive/KaggleAMEX/Data/spend_variables.parquet'
    new = pd.read_parquet(dir)
    new = new.loc[customers]
    df  = pd.concat([new, df], axis=1)
    print('Done spend variables', df.shape)
    
    # Balance variables 
    dir = '/content/drive/MyDrive/KaggleAMEX/Data/balance_features.parquet'
    new = pd.read_parquet(dir)
    new = new.loc[customers]
    df  = pd.concat([new, df], axis=1)
    print('Done balance variables', df.shape)
    
    # Delinquency variables 
    dir = '/content/drive/MyDrive/KaggleAMEX/Data/delinquency_features.parquet'
    new = pd.read_parquet(dir)
    new = new.loc[customers]
    df  = pd.concat([new, df], axis=1)
    print('Done delinquency variables', df.shape)
    
    # count na values
    dir = '/content/drive/MyDrive/KaggleAMEX/Data/na_dummies.parquet'
    new = pd.read_parquet(dir)
    new = new.loc[customers]
    df  = pd.concat([new, df], axis=1)
    print('Done count na variables', df.shape)
    
    # category changes for cat vars
    dir = '/content/drive/MyDrive/KaggleAMEX/Data/cat_changes.parquet'
    new = pd.read_parquet(dir)
    new = new.loc[customers]
    df  = pd.concat([new, df], axis=1)
    print('Done cat change variables', df.shape)

    # 'last' - 'mean' 
    dir = '/content/drive/MyDrive/KaggleAMEX/Data/last_m_mean.parquet'
    new = pd.read_parquet(dir)
    new = new.loc[customers]
    df  = pd.concat([new, df], axis=1)
    print('Done last - mean variables', df.shape)
    
    # First difference of last values 
    dir = '/content/drive/MyDrive/KaggleAMEX/Data/first_diff.parquet'
    new = pd.read_parquet(dir)
    new = new.loc[customers]
    df  = pd.concat([new, df], axis=1)
    print('Done diff_1 variables', df.shape)
    
    '''
    # Second difference of last values 
    dir = '/content/drive/MyDrive/KaggleAMEX/Data/second_diff.parquet'
    new = pd.read_parquet(dir) 
    df  = pd.concat([new, df], axis=1).reset_index(drop=True)

    # numerical 'last' - 'first'
    dir = '/content/drive/MyDrive/KaggleAMEX/Data/first_last.parquet'
    new = pd.read_parquet(dir)
    df  = pd.concat([new, df], axis=1).reset_index(drop=True)
    '''

    # Add targets
    targets      = pd.read_csv(targets_dir, usecols = ['target']).astype('int8')
    df['target'] = targets.loc[customers]

    # Reset index
    df.reset_index(drop=True, inplace=True)

    return df

# Read & Transform Data 

In [ ]:
train = process_and_feature_engineer([2])

Done base (6098, 957)
Done payment variables (6098, 1050)
Done risk variables (6098, 1110)
Done spend variables (6098, 1310)
Done balance variables (6098, 1743)
Done delinquency variables (6098, 2055)
Done count na variables (6098, 2244)
Done cat change variables (6098, 2287)
Done last - mean variables (6098, 2432)
Done diff_1 variables (6098, 2578)


In [ ]:
# drop cold with only one value
to_drop = [col for col in train.columns if len(train[col].unique()) == 1]
print(len(to_drop))
train.drop(to_drop, axis=1, inplace=True)

118


In [ ]:
features = pd.read_csv('/content/drive/MyDrive/KaggleAMEX/Models/freq_2/features.csv', index_col=0)

In [ ]:
train = train[list(features.index.values[:-1]) + ['target']]

In [ ]:
params = pd.read_csv('/content/drive/MyDrive/KaggleAMEX/Models/freq_2/parameters.csv', index_col=0)

In [ ]:
params.to_dict()['0']

{'alpha': '1',
 'booster': 'gbtree',
 'colsample_bylevel': '0.8',
 'colsample_bynode': '0.7',
 'colsample_bytree': '0.8',
 'gamma': '0.5',
 'lambda': '2.8',
 'learning_rate': '0.012',
 'max_bin': '180',
 'max_depth': '30',
 'max_leaves': '30',
 'min_child_weight': '2',
 'objective': 'binary:logistic',
 'predictor': 'gpu_predictor',
 'sampling_method': 'gradient_based',
 'scale_pos_weight': '1.5',
 'seed': '42',
 'subsample': '0.6',
 'tree_method': 'gpu_hist'}

# Extreme Gradient Boost

## Custom metric

In [ ]:
# This is the metric given by rules
def amex_metric_mod(y_true, y_pred):

    labels     = np.transpose(np.array([y_true, y_pred]))
    labels     = labels[labels[:, 1].argsort()[::-1]]
    weights    = np.where(labels[:,0]==0, 20, 1)
    cut_vals   = labels[np.cumsum(weights) <= int(0.04 * np.sum(weights))]
    top_four   = np.sum(cut_vals[:,0]) / np.sum(labels[:,0])

    gini = [0,0]
    for i in [1,0]:
        labels         = np.transpose(np.array([y_true, y_pred]))
        labels         = labels[labels[:, i].argsort()[::-1]]
        weight         = np.where(labels[:,0]==0, 20, 1)
        weight_random  = np.cumsum(weight / np.sum(weight))
        total_pos      = np.sum(labels[:, 0] *  weight)
        cum_pos_found  = np.cumsum(labels[:, 0] * weight)
        lorentz        = cum_pos_found / total_pos
        gini[i]        = np.sum((lorentz - weight_random) * weight)


    return 0.5 * (gini[1]/gini[0] + top_four)
    

# Metric for XGB
def xgboost_amex_metric_mod(predt: np.ndarray, dtrain: xgb.DMatrix):
    y = dtrain.get_label()
    return 'AMEXcustom', 1-amex_metric_mod(y, predt)

def permutation_importance(train, valid_idx, features, model, kag_mets):

    # Permutation importance
    x_test  = train.loc[valid_idx, features]
    y_test  = train.loc[valid_idx, 'target']
    metrics = []

    for column, feature in enumerate(features):

          #Save original feature
          original_feature = x_test[feature].copy()
          metric = []

          for i in range(2):

              # Shuffel
              x_test[feature] = np.random.permutation(original_feature)

              #Build DMatrix
              dvalid = xgb.DeviceQuantileDMatrix(data=cupy.array(x_test), label=y_test)
              
              #Predict
              preds  = model.predict(dvalid, iteration_range=(0, model.best_iteration + 1))
              metric.append(kag_mets - amex_metric_mod(y_test, preds))

          metrics.append(np.mean(metric))

          #Rebuild data
          x_test[feature] = original_feature
    
    return metrics

## Grid Search & Permutation Importance

In [ ]:
learning_rate     = [0.011, 0.012, 0.013]          
gamma             = [0.4, 0.5, 0.6]              
max_depth         = [28, 30, 32]             
min_child_weight  = [1.9, 2, 2.1]             
max_leaves        = [28, 30, 32]             
max_bin           = [170, 180, 190]               

subsample         = [0.5, 0.6]            
sampling_method   = ['gradient_based']      

colsample_bytree  = [0.8]             
colsample_bylevel = [0.8]              
colsample_bynode  = [0.6, 0.7]              
                   
alpha             = [0.9, 1, 1.1]             
lambda_           = [2.7, 2.8, 2.9]             

scale_pos_weight  = [1.45, 1.5, 1.55]              

tree_method       = ['gpu_hist']        
objective         = ['binary:logistic']
predictor         = ['gpu_predictor']
booster           = ['dart']
seed              = [22,42,11]
sample_type       = ['uniform', 'weighted']
normalize_type    = ['tree', 'forest']
rate_drop         = [0.1, 0.2, 0.3]
skip_drop         = [0.5, 0.6, 0.7]

# Get Grids for xgb
hyper_parms = ['learning_rate', 'gamma', 'max_depth', 'min_child_weight', 'subsample', 'sampling_method', 'colsample_bytree', 'colsample_bylevel', 'colsample_bynode', 'lambda', 'alpha', 'scale_pos_weight', 'max_leaves', 'max_bin', 'tree_method', 'objective', 'predictor', 'seed', 'booster', 'sample_type', 'normalize_type', 'rate_drop', 'skip_drop']
variants    = list(itertools.product(learning_rate, gamma, max_depth, min_child_weight, subsample, sampling_method, colsample_bytree, colsample_bylevel, colsample_bynode, lambda_, alpha, scale_pos_weight, max_leaves, max_bin, tree_method, objective, predictor, seed, booster, sample_type, normalize_type, rate_drop, skip_drop))
xgb_grid    = [dict(zip(hyper_parms, variant)) for variant in variants]
random.shuffle(xgb_grid)
random.shuffle(xgb_grid)
xgb_grid = xgb_grid[:200]

print(f'There are {len(xgb_grid)} different Models.', '\n', '\n')

There are 200 different Models. 
 



In [ ]:
# Out-of-fold predictions and feature importances
oof_cols    = [f'fold_{i}' for i in range(FOLDS)] + ['avg', 'oof']
oof         = pd.DataFrame(columns = oof_cols)
importances = pd.DataFrame()
premu_imp   = pd.DataFrame(index = train.columns[:-1])

for model_num, xgb_parms in enumerate(xgb_grid):

    kag_mets         = []
    oof_model        = pd.DataFrame() 
    importance_model = pd.DataFrame(index = [f'f{num}' for num in range(train.shape[1]-1)])

    skf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=SEED)
    for fold, (train_idx, valid_idx) in enumerate(skf.split(train, train.target)):
        
        print(f'############### Model {model_num} Fold {fold} ##################')
        print('##################################################')
        
        # Train and valid set for given fold       
        x_valid = cupy.array(train.loc[valid_idx, train.columns[:-1]])
        y_valid = cupy.array(train.loc[valid_idx, 'target'])
        dvalid  = xgb.DMatrix(data=x_valid, label=y_valid)
         
        x_train = cupy.array(train.loc[train_idx, train.columns[:-1]])
        y_train = cupy.array(train.loc[train_idx, 'target'])
        dtrain  = xgb.DeviceQuantileDMatrix(data=x_train, label=y_train)

        # Train model fold 'k'
        model = xgb.train(params                = xgb_parms,
                          dtrain                = dtrain,
                          evals                 = [(dvalid,'valid')],
                          custom_metric         = xgboost_amex_metric_mod,
                          num_boost_round       = 10000,
                          early_stopping_rounds = 200,
                          verbose_eval          = 500)
        
        # Save model
        #model.save_model(f'{model_dir}/freq_1_fold_{fold}.xgb')
                
        # Validation - Infer out-of-fold
        preds     = model.predict(dvalid, iteration_range=(0, model.best_iteration + 1))
        df        = pd.DataFrame(data=preds, index=valid_idx, columns=['oof'])
        oof_model = pd.concat([oof_model, df], axis = 0)

        # Kaggle metric
        kag_mets.append(amex_metric_mod(y_valid.get(), preds))
        curr_best = np.max([oof[f'fold_{fold}'].max(), kag_mets[-1]])
        print(f'Metric = {kag_mets[-1]}, Best = {curr_best}', '\n')
        
        # Skip if no significant improvment
        if kag_mets[-1] + 0.015 < curr_best: break
        
        # Permutation importance
        premu_imp[f'grid_{model_num}_fold_{fold}'] = permutation_importance(train, valid_idx, train.columns[:-1], model, kag_mets[-1])
        
        # Clean RAM & GPU RAM
        del y_train, x_train, y_valid, x_valid, dtrain, dvalid, train_idx, valid_idx
        mempool.free_all_blocks(), pinned_mempool.free_all_blocks(), gc.collect()
  
    # Skip if fold not compleat
    if len(kag_mets) != 5: continue

    # Metrics
    kag_mets.append(np.mean(kag_mets))
    kag_mets.append(amex_metric_mod(train.target.values, oof_model.sort_index().oof.values))
    oof.loc[model_num] = kag_mets
    print(f'avg: {kag_mets[-2]}, Max: {oof.avg.max()}')
    print(f'oof: {kag_mets[-1]}, Max: {oof.oof.max()}', '\n', '\n', '\n')

# Average over folds
premu_imp['mean'] = premu_imp.mean(axis=1)

############### Model 0 Fold 0 ##################
##################################################
[0]	valid-logloss:0.68842	valid-AMEXcustom:0.52791
[500]	valid-logloss:0.38589	valid-AMEXcustom:0.38217
[648]	valid-logloss:0.37891	valid-AMEXcustom:0.38146
Metric = 0.6285683595570538, Best = nan 

############### Model 0 Fold 1 ##################
##################################################
[0]	valid-logloss:0.68861	valid-AMEXcustom:0.56074
[500]	valid-logloss:0.39239	valid-AMEXcustom:0.42526
[1000]	valid-logloss:0.38404	valid-AMEXcustom:0.39178
[1267]	valid-logloss:0.38524	valid-AMEXcustom:0.40346
Metric = 0.6133459478722374, Best = nan 

############### Model 0 Fold 2 ##################
##################################################
[0]	valid-logloss:0.68873	valid-AMEXcustom:0.63992
[500]	valid-logloss:0.39914	valid-AMEXcustom:0.43596
[1000]	valid-logloss:0.39163	valid-AMEXcustom:0.42665
[1164]	valid-logloss:0.39210	valid-AMEXcustom:0.42790
Metric = 0.5781683537759641, Bes

In [ ]:
# 13 & not 13
model_metric

0.7931844627660678

In [ ]:
params = pd.DataFrame.from_dict(xgb_grid[0], orient='index')
params.to_csv(f'{model_dir}/parameters.csv')

# Feature Importance

https://towardsdatascience.com/be-careful-when-interpreting-your-features-importance-in-xgboost-6e16132588e7

Gain - The relative contribution of the corresponding feature to the model. This is calculated by taking each feature’s contribution for each tree in the model. A higher value of this metric when compared to another feature implies it is more important for generating a prediction.

Coverage - The relative number of observations related to this feature. For example, if you have 100 observations, 4 features and 3 trees, and suppose feature1 is used to decide the leaf node for 10, 5, and 2 observations in tree1, tree2 and tree3 respectively; then the metric will count cover for this feature as 10+5+2 = 17 observations. This will be calculated for all the 4 features and the cover will be 17 expressed as a percentage for all features’ cover metrics.

Weight - The percentage representing the relative number of times a particular feature occurs in the trees of the model. In the above example, if feature1 occurred in 2 splits, 1 split and 3 splits in each of tree1, tree2 and tree3; then the weight for feature1 will be 2+1+3 = 6. The frequency for feature1 is calculated as its percentage weight over weights of all features.

In [ ]:
importance_types = ['cover', 'gain', 'weight', 'total_gain', 'total_cover']

In [ ]:
df = model.get_score(importance_type='cover')
df = pd.DataFrame.from_dict(df, orient='index', columns = ['cover'])

In [ ]:
df = model.get_score(importance_type='gain')
df = pd.DataFrame.from_dict(df, orient='index', columns = ['gain'])